# Extract data for urban calculations

Test input for Tanzania

0. Select focal ADM, buffer by 1km, rasterize as [0, 1]
1. Download DEM data from ASTER, mosaick
2. Calculate slope of DEM
3. Extract water layer from Globcover
4. Rasterize building footprints
5. Select population layer
6. Standardize all rasters to population layer
   a. Set area outside of focal admin to NoData
   b. Set everything to 16bit
   


In [5]:
import sys, os, importlib, shutil
import requests
import rasterio, elevation, richdem
import rasterio.warp
from rasterio import features

import pandas as pd
import geopandas as gpd
import numpy as np

#Import GOST urban functions
sys.path.append("../")
import src.UrbanRaster as urban

#Import raster helpers
sys.path.append("../../gostrocks/src")

import GOSTRocks.rasterMisc as rMisc
from GOSTRocks.misc import tPrint

In [6]:
global_bounds = "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
global_landcover  = "/home/public/Data/GLOBAL/LANDCOVER/GLOBCOVER/2015/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7.tif"
global_ghspop = "/home/public/Data/GLOBAL/Population/GHS/250/GHS_POP_E2015_GLOBE_R2019A_54009_250_V1_0.tif"
global_ghbuilt = "/home/public/Data/GLOBAL/URBAN/GHS/GHS_1K_BUILT/GHS_BUILT_LDS2014_GLOBE_R2018A_54009_1K_V1_0.tif"
inG = gpd.read_file(global_bounds)
iso3 = 'GHA'

#Define paths to local population files
local_path = "/home/public/Data/COUNTRY/{country}/POPULATION/WORLDPOP/".format(country=iso3)
#pop_2015_con = os.path.join(local_path, "ppp_prj_2015_%s_UNadj.tif" % iso3)
pop_2018_con = os.path.join(local_path, "%s_ppp_2020_UNadj_constrained.tif" % iso3.lower())
pop_2015_un = os.path.join(local_path, "%s_ppp_2015_UNadj.tif" % iso3.lower())
pop_2018_un = os.path.join(local_path, "%s_ppp_2018_UNadj.tif" % iso3.lower())

inD = inG.loc[inG['ISO3'] == iso3]
inD['geometry'] = inD['geometry'].apply(lambda x: x.buffer(500))
inD = inD.to_crs({'init':'epsg:4326'})

output_folder = "/home/wb411133/temp/%s_URBAN_DATA" % iso3
final_folder = os.path.join(output_folder, "FINAL_STANDARD")
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    os.makedirs(final_folder)
    
dem_file = os.path.join(output_folder, "%s_DEM.tif" % iso3)
slope_file = os.path.join(output_folder, "%s_SLOPE.tif" % iso3)
lc_file = os.path.join(output_folder, "%s_LC.tif" % iso3)
lc_file_h20 = os.path.join(output_folder, "%s_LC_H20.tif" % iso3)
ghspop_file = os.path.join(output_folder, "%s_GHS.tif" % iso3)
ghsbuilt_file = os.path.join(output_folder, "%s_GHSBUILT.tif" % iso3)
admin_file  =  os.path.join(output_folder, "%s_ADMIN.tif" % iso3)
admin_shp  =  os.path.join(final_folder, "%s_ADMIN.shp" % iso3)

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [7]:
# Write shapefile to file
if not os.path.exists(admin_shp):
    inD.to_file(admin_shp)

In [8]:
# Download DEM
importlib.reload(elevation)
if not os.path.exists(dem_file):
    tPrint("Downloading DEM")
    #elevation.clip(bounds=inD.total_bounds, max_download_tiles=90000, output=dem_file, product='SRTM3')

# Calculate slope
if not os.path.exists(slope_file) and os.path.exists(dem_file):
    tPrint("Calculating slope")
    in_dem = rasterio.open(dem_file)
    in_dem_data = in_dem.read()
    beau  = richdem.rdarray(in_dem_data[0,:,:], no_data=in_dem.meta['nodata'])
    slope = richdem.TerrainAttribute(beau, attrib='slope_riserun')
    meta = in_dem.meta.copy()
    meta.update(dtype = slope.dtype)
    with rasterio.open(slope_file, 'w', **meta) as outR:
        outR.write_band(1, slope)
    
# Extract water from globcover
if not os.path.exists(lc_file_h20):
    tPrint("Extracting water")
    if not os.path.exists(lc_file):
        rMisc.clipRaster(rasterio.open(global_landcover), inD, lc_file)
    in_lc = rasterio.open(lc_file)
    inL = in_lc.read()
    lcmeta = in_lc.meta.copy()
    tempL = (inL == 210).astype(lcmeta['dtype'])

    lcmeta.update(nodata=255)

    with rasterio.open(lc_file_h20, 'w', **lcmeta) as out:
        out.write(tempL)

    os.remove(lc_file)

#Extract GHS-Pop
if not os.path.exists(ghspop_file):
    tPrint("Extracting GHS-POP")
    rMisc.clipRaster(rasterio.open(global_ghspop), inD, ghspop_file)

#Extract GHS-Built
if not os.path.exists(ghsbuilt_file):
    tPrint("Clipping GHS-Built")
    rMisc.clipRaster(rasterio.open(global_ghbuilt), inD, ghsbuilt_file)
    
#Rasterize admin boundaries
if not os.path.exists(admin_file):
    tPrint("Rasterizing admin boundaries")
    xx = rasterio.open(ghspop_file)
    res = xx.meta['transform'][0]
    tempD = inD.to_crs(xx.crs)
    shapes = ((row['geometry'], 1) for idx, row in tempD.iterrows())
    burned = features.rasterize(shapes=shapes, out_shape=xx.shape, fill=0, transform=xx.meta['transform'], dtype='int16')
    meta = xx.meta.copy()
    meta.update(dtype=burned.dtype)
    with rasterio.open(admin_file, 'w', **meta) as outR:
        outR.write_band(1, burned)

07:47:04	Downloading DEM


# Raster standardization

In [ ]:
ghs_R = rasterio.open(ghspop_file)    
out_array = ghs_R.read() * 0
#Read in admin data and get nodata area
in_admin = rasterio.open(admin_file)
in_a = in_admin.read()
in_a_mask = in_a == 0

In [ ]:
if not os.path.exists(final_folder):
    os.makedirs(final_folder)
for file_def in [
        #file, type, scale values
        #[pop_2015_con, 'N', True],
        [pop_2018_con, 'N', True],
        [pop_2015_un, 'N', True],
        [pop_2018_un, 'N', True],
        [ghspop_file, 'N', True],
        [lc_file_h20, 'C', False],
        #[slope_file, 'N', False],
        #[dem_file, 'N', False],
        [ghsbuilt_file, 'N', False]        
    ]:
    print(file_def[0])
    out_file = os.path.join(final_folder, os.path.basename(file_def[0]))    
    # scale and project file to GHS pop
    if not os.path.exists(out_file):
        out_array = ghs_R.read() * 0
        in_raster = rasterio.open(file_def[0])
        in_r = in_raster.read()
        rSample = rasterio.warp.Resampling.cubic
        if file_def[1] == 'C':
            rSample = rasterio.warp.Resampling.nearest
        rasterio.warp.reproject(in_r, out_array, 
                                src_transform=in_raster.meta['transform'], dst_transform=ghs_R.meta['transform'],
                                src_crs = in_raster.crs, dst_crs = ghs_R.crs,
                                src_nodata = in_raster.meta['nodata'], dst_nodata = ghs_R.meta['nodata'],
                               resample = rSample)
        out_array[out_array == ghs_R.meta['nodata']] = 0.
        # If values are to be scaled based on area change, do it here
        if file_def[2]:
            #Determine scale difference between rasters
            width_ratio = in_raster.shape[0] / ghs_R.shape[0]
            height_ratio = in_raster.shape[1] / ghs_R.shape[1]
            total_ratio = width_ratio * height_ratio
            out_array = out_array * total_ratio
            out_array[out_array < 0] = ghs_R.meta['nodata']
        # Set area outside national boundaries to nodata
        out_array[in_a_mask] = ghs_R.meta['nodata']
        out_meta = ghs_R.meta.copy()
        out_meta.update(nodata=ghs_R.meta['nodata'])
        with rasterio.open(out_file, 'w', **out_meta) as outR:
            outR.write(out_array)
    # Write no data layers to file
    out_no_data_file = os.path.join(final_folder, "NO_DATA_%s" % os.path.basename(file_def[0]))
    if not os.path.exists(out_no_data_file):
        out_array = ghs_R.read() * 0
        in_raster = rasterio.open(file_def[0])
        in_r = in_raster.read()
        # create binary file defining no data area
        in_r = (in_r == in_raster.meta['nodata']).astype(ghs_R.meta['dtype'])
        rasterio.warp.reproject(in_r, out_array, 
                                src_transform=in_raster.meta['transform'], dst_transform=ghs_R.meta['transform'],
                                src_crs = in_raster.crs, dst_crs = ghs_R.crs,
                                src_nodata = in_raster.meta['nodata'], dst_nodata = ghs_R.meta['nodata'],
                                resample = rasterio.warp.Resampling.nearest)
        out_meta = ghs_R.meta.copy()
        with rasterio.open(out_no_data_file, 'w', **out_meta) as outR:
            outR.write(out_array)

# Calculate urban extents

In [ ]:
importlib.reload(urban)
# Calculate urban extents from population layers
for p_file in [pop_2015_un, pop_2018_un, ghspop_file]: #pop_2015_con, pop_2018_con, 
    final_pop = os.path.join(final_folder, os.path.basename(p_file))
    final_urban    = final_pop.replace(".tif", "_urban.tif")
    final_urban_hd = final_pop.replace(".tif", "_urban_hd.tif")
    urbanR = urban.urbanGriddedPop(final_pop)
    tPrint(final_urban)
    #calculate density values
    in_raster = rasterio.open(p_file)
    width_ratio = in_raster.shape[0] / ghs_R.shape[0]
    height_ratio = in_raster.shape[1] / ghs_R.shape[1]
    total_ratio = width_ratio * height_ratio
    if not os.path.exists(final_urban):
        urban_shp   = urbanR.calculateUrban(densVal= (3 * total_ratio), totalPopThresh=5000,  raster=final_urban)
    tPrint(final_urban_hd)
    if not os.path.exists(final_urban_hd):
        cluster_shp = urbanR.calculateUrban(densVal=(15 * total_ratio), totalPopThresh=50000, raster=final_urban_hd)

# Check NoData values in final datasets

In [ ]:
tiff_files = []
for root, dirs, files in os.walk(final_folder):
    for f in files:
        if f[-4:] == ".tif":
            tiff_files.append(os.path.join(final_folder, f))

for tFile in tiff_files:
    xx = rasterio.open(tFile)
    total_shape = xx.shape[1] * xx.shape[0]
    xx_data = xx.read()
    total_no_data = (xx_data == xx.meta['nodata']).sum()
    print("%s\t%s\t%s" % (total_shape, total_no_data, os.path.basename(tFile)))

In [ ]:
xx.shape

# Deburgging

In [ ]:
xx = rasterio.open(pop_2015_con)
inD = xx.read()

In [ ]:
print(inD.shape[1] * inD.shape[2])
print((inD > 0).sum())
print((inD > 3).sum())
print((inD > 15).sum())

In [ ]:
print(inD[(inD > 0)].sum())
print(inD[(inD > 3)].sum())
print(inD[(inD > 15)].sum())
